In [2]:
from google.colab import auth
from google.oauth2 import service_account
import google.auth  # Import google.auth

def authenticate_service():
    auth.authenticate_user()
    credentials, project = google.auth.default()
    return service_account.Credentials.from_service_account_file('/content/amazing-capsule-448912-a5-f10513e685de.json')

creds = authenticate_service()

In [3]:
from googleapiclient.discovery import build

def create_google_doc(creds):
    service = build('docs', 'v1', credentials=creds)
    doc = {
        'title': 'Product Team Sync'
    }
    doc = service.documents().create(body=doc).execute()
    return doc['documentId']

document_id = create_google_doc(creds)

In [4]:
def make_document_public(document_id):
        """Makes a Google Doc publicly accessible.

        Args:
            document_id: The ID of the document to make public.
        """
        service = build('drive', 'v3', credentials=creds)
        permission = {
            'type': 'anyone',
            'role': 'reader'  # or 'writer' for editing access
        }
        service.permissions().create(fileId=document_id, body=permission).execute()

make_document_public(document_id)
print(document_id)

18SKEiQlgjuuBNinVwW2pjHRzJt7d80mfIfXUuOnVr1Y


In [9]:
from googleapiclient.discovery import build
import re

# Authenticate and create Google Docs service
service = build('docs', 'v1', credentials=creds)

# Function to get end index of the doc
def get_end_index(document_id):
    doc = service.documents().get(documentId=document_id).execute()
    return doc['body']['content'][-1]['endIndex']-1

# Function to apply style
def apply_style(document_id, text, style_type):
    end_index = get_end_index(document_id)
    requests = [
        {
            'insertText': {
                'location': {
                    'index': end_index,
                },
                'text': text + '\n'
            }
        },
        {
            'updateTextStyle': {
                'range': {
                    'startIndex': end_index,
                    'endIndex': end_index + len(text) + 1,
                },
                'textStyle': {
                    'bold': style_type.startswith('Heading'),
                    'fontSize': {
                        'magnitude': 24 if style_type == 'Heading 1' else (
                            20 if style_type == 'Heading 2' else 16),
                        'unit': 'PT'
                    }
                },
                'fields': 'bold,fontSize'
            }
        }
    ]
    service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()

# Function to add checkboxes or bullet points
def add_checkboxes_or_bullets(document_id, text, is_checkbox=False):
    end_index = get_end_index(document_id)
    text = text.strip()

    requests = [
        {
            'insertText': {
                'location': {
                    'index': end_index,
                },
                'text': text + '\n'
            }
        }
    ]

    if is_checkbox:
        requests.append({
            'createParagraphBullets': {
                'range': {
                    'startIndex': end_index,
                    'endIndex': end_index + 1
                },
                'bulletPreset': 'BULLET_CHECKBOX'
            }
        })
    else:
        # Convert the inserted text to a list item
        requests.append({
            'createParagraphBullets': {
                'range': {
                    'startIndex': end_index,
                    'endIndex': end_index + len(text) + 1,  # Include newline
                },
                'bulletPreset': 'BULLET_ARROW_DIAMOND_DISC',  # Bullet style
            }
        })

    mentions = find_mentions(text)
    if mentions != None and len(mentions) > 1:
        for start, end in mentions:
            requests.append(add_mention(document_id, start, end))
    service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()

def find_mentions(text):
  if not isinstance(text, str):
      return None
  mentions = []
  for match in re.finditer(r"@\S+\b", text): # \S+ for non-whitespace characters
      start_index = match.start()
      end_index = match.end()
      mentions.append((start_index, end_index))
  return mentions

# Function to add text with mention styling
def add_mention(document_id, start, end):
    return {
            'updateTextStyle': {
                'range': {
                    'startIndex': start,
                    'endIndex': end,
                },
                'textStyle': {
                    'bold': True,
                    'foregroundColor': {
                        'color': {
                            'rgbColor': {
                                'red': 0.0,
                                'green': 0.0,
                                'blue': 1.0
                            }
                        }
                    }
                },
                'fields': 'bold,foregroundColor'
            }
        }

def add_nestedList(document_id, my_list):
    indent_corr = 1
    start_index = get_end_index(document_id)
    text_insert = ""
    deleteParagraphBullets = []
    for text in my_list:
        if text.startswith('* '):
            text_insert += text[2:] + '\n'
        elif text.startswith('  * '):
            text_insert += '\t' + text[4:] + '\n'
        elif text.startswith('    * '):
            text_insert += '\t\t' + text[6:] + '\n'
        else:
            text_normal = text + '\n'
            text_insert += text_normal
            end_index_normal = start_index + len(text_insert) + 1 - indent_corr
            start_index_normal = end_index_normal - len(text_normal)
            deleteParagraphBullets.append({
                "deleteParagraphBullets": {
                    'range': {
                        'startIndex': start_index_normal,
                        'endIndex': start_index_normal + 1,
                    },
                }
            })
    deleteParagraphBullets.reverse()
    indented_requests = [
        {
            "insertText": {
                "text": text_insert,
                'location': {'index': start_index},
            }
        },
        {
            "createParagraphBullets": {
                'range': {
                    'startIndex': start_index,
                    'endIndex': start_index + len(text_insert),
                },
                "bulletPreset": "NUMBERED_DECIMAL_ALPHA_ROMAN",
            }
        }
    ] + deleteParagraphBullets
    service.documents().batchUpdate(documentId=document_id, body={'requests': indented_requests}).execute()



# Function to clear document content
def clear_document_content(document_id):
    requests = [
        {
            'deleteContentRange': {
                'range': {
                    'startIndex': 1,
                    'endIndex': get_end_index(document_id) - 1
                }
            }
        }
    ]
    service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()

# Function to delete document
def delete_document(document_id):
    drive_service = build('drive', 'v3', credentials=creds)
    drive_service.files().delete(fileId=document_id).execute()


In [7]:
def read_local_markdown_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

file_path = '/content/markdown.md'
markdown_content = read_local_markdown_file(file_path)

In [11]:
import time
def process_markdown_content(markdown_content, document_id):
  lines = markdown_content.split('\n')
  isNested = False
  nestedList = []

  for line in lines:
    time.sleep(10)
    if re.match(r'\s*\* ', line):
      if not isNested:
        nestedList.append(line)
        isNested = True
        print("Entering in nested list.")
      elif isNested:
        nestedList.append(line)
    elif isNested:
      add_nestedList(document_id, nestedList)
      nestedList = []
      isNested = False
      print("Exiting nested list.")
      apply_style(document_id, line, 'Normal')
    else:
      if line.startswith('# '):
        apply_style(document_id, line[2:], 'Heading 1')
      elif line.startswith('## '):
        apply_style(document_id, line[3:], 'Heading 2')
      elif line.startswith('### '):
        apply_style(document_id, line[4:], 'Heading 3')
      elif line.startswith('- [ ] '):
        add_checkboxes_or_bullets(document_id, line[6:], is_checkbox=True)
      elif line.startswith('-'):
        add_checkboxes_or_bullets(document_id, line[2:], is_checkbox=False)
      elif '@' in line:
        add_mention(document_id, line)
      # Add other condition checks as necessary
      else:
        apply_style(document_id, line, 'Normal')

# Fetch and process the markdown content
process_markdown_content(markdown_content, document_id)
print('Done parsing and updating...')

Entering in nested list.
Exiting nested list.
Entering in nested list.
Exiting nested list.
Entering in nested list.
Exiting nested list.
Entering in nested list.
Exiting nested list.
Entering in nested list.
Exiting nested list.
Done parsing and updating...
